import libraries

In [9]:
import tmdbsimple as tmdb
import requests
import pandas as pd
import time
from ast import literal_eval
import imdb

Skip the next 2 cells, as they only need to be run once.

In [4]:
key = open('key.txt','r').read()
payload = "{}"

#this cell only needs to be run once
"""
movie_df = pd.DataFrame() #create empty dataframe to enable 'while loop' below

page=1
while movie_df.shape[0] < 1000:
    url = "https://api.themoviedb.org/3/movie/top_rated?api_key={0}&language=en-US&page={1}".format(key, str(page))
    response = requests.request("GET", url, data=payload).json()
    if page == 1: #initialize dataframe on first loop
        movie_df = pd.DataFrame(response['results'])
    else:
        movie_df = movie_df.append(pd.DataFrame(response['results']))
    
    movie_df = movie_df[movie_df['original_language']=='en'] #remove non english movies
    time.sleep(0.25) #rate limit is 4 pages per second
    page+=1
    
movie_df.reset_index(inplace=True,drop=True) #reset index since we dropped non english rows

#drop irrelevant columns for this analysis
dropCols = ['adult','backdrop_path', 'original_language','original_title', 'poster_path','video']

movie_df.drop(dropCols,axis=1,inplace=True)

"""

**IMDB Mapping**

In [84]:
def tmdb_to_imdb(tmdb_id):
    time.sleep(0.25) #rate limit is 4 requests per second
    url = "https://api.themoviedb.org/3/movie/{0}/external_ids?api_key={1}&language=en-US".format(tmdb_id, key)
    response = requests.request("GET", url, data=payload).json()
    if 'imdb_id' in response:
        return response['imdb_id']
    else:
        return None


In [119]:
#this cell only has to be run once
"""
movie_df['imdb_id'] = movie_df['id'].apply(lambda x: tmdb_to_imdb(x))
movie_df.to_csv('data\movie_df.csv',encoding="utf-8",index=False)
"""

In [2]:
movie_df = pd.read_csv('data\movie_df.csv',encoding="utf-8",converters={'genre_ids':literal_eval})

In [120]:
movie_df.head()

,genre_ids,id,overview,popularity,release_date,title,vote_average,vote_count,imdb_id
0,"[18, 80]",278,Framed in the 1940s for the double murder of h...,28.527767,1994-09-23,The Shawshank Redemption,8.5,9773,tt0111161
1,"[18, 80]",238,"Spanning the years 1945 to 1955, a chronicle o...",36.965452,1972-03-14,The Godfather,8.5,7394,tt0068646
2,"[18, 36, 10752]",424,The true story of how businessman Oskar Schind...,19.945455,1993-11-29,Schindler's List,8.4,5518,tt0108052
3,"[18, 80]",240,In the continuing saga of the Corleone crime f...,30.191804,1974-12-20,The Godfather: Part II,8.4,4249,tt0071562
4,"[18, 9648]",452522,Standalone version of the series pilot with an...,5.969249,1989-12-31,Twin Peaks,8.4,123,tt0278784


Now we can use the imdb_id to get the imdb attributes for each movie

In [10]:
imbd_object = imdb.IMDb()

**Genres**

In [7]:
url = "https://api.themoviedb.org/3/genre/movie/list?api_key={0}&language=en-US".format(key)
response = requests.request("GET", url, data=payload).json()

genreDict = dict(zip([i['id'] for i in response['genres']],
                     [i['name'] for i in response['genres']]))

genreDict

{12: 'Adventure',
 14: 'Fantasy',
 16: 'Animation',
 18: 'Drama',
 27: 'Horror',
 28: 'Action',
 35: 'Comedy',
 36: 'History',
 37: 'Western',
 53: 'Thriller',
 80: 'Crime',
 99: 'Documentary',
 878: 'Science Fiction',
 9648: 'Mystery',
 10402: 'Music',
 10749: 'Romance',
 10751: 'Family',
 10752: 'War',
 10770: 'TV Movie'}